In [1]:
import openpyxl
import csv
import json
import requests
import re
import pandas as pd
import time
from time import sleep
import pyautogui as pag
from bs4 import BeautifulSoup
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver import Edge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchAttributeException
from selenium.webdriver.support.ui import Select

In [7]:
options = uc.ChromeOptions()
options.headless = False
options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(version_main=135)
driver.get("https://www.agribank.com.vn/vn/atm-chi-nhanh")

In [8]:
select_elem = Select(driver.find_element("id", "cityBranch"))

In [9]:
csv_file = open('AGRIBANK(PGD+ATM).csv', mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Province', 'Name', 'Address', 'Latitude', 'Longitude'])

42

In [10]:
for index in range(1, len(select_elem.options)):
    province_name = select_elem.options[index].text.strip()
    print(f"\n👉 Đang xử lý tỉnh: {province_name}")

    # Chọn tỉnh/thành
    select_elem.select_by_index(index)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    branches = soup.find_all('div', class_='atm_cn')

    for div in branches:
        onclick = div.get('onclick')
        if not onclick:
            continue

        match = re.search(r'gotoLocation\(([\d.]+),([\d.]+),\'(.+?)\'\)', onclick)
        if match:
            lat = float(match.group(1))
            lng = float(match.group(2))
            raw_info = match.group(3)

            info_soup = BeautifulSoup(raw_info, 'html.parser')
            name = info_soup.find('b').text if info_soup.find('b') else 'Không rõ'
            address = info_soup.get_text().split('\n')[-1]

            print(f"- {name} | {address} | ({lat}, {lng})")

            # Ghi vào CSV
            csv_writer.writerow([province_name, name, address, lat, lng])

driver.quit()
csv_file.close()
print("\n✅ Đã lưu xong dữ liệu vào file vietcombank_branches.csv")


👉 Đang xử lý tỉnh: Hà Nội
- Chi nhánh Sở giao dịch | Chi nhánh Sở giao dịchSố 2 và số 4 Láng Hạ, Phường Thành Công, Quận Ba Đình, Thành phố Hà Nội | (21.022833, 105.818854)
- Chi nhánh Sở giao dịch - Phòng giao dịch số 1 | Chi nhánh Sở giao dịch - Phòng giao dịch số 1Tầng 1 và tầng 2 - K17.TT1 - SH24, Khu trung tâm khu đô thị Tây Hồ Tây, Phường Xuân La, Quận Tây Hồ, Thành phố Hà Nội | (21.05883, 105.79307)
- Chi nhánh Sở giao dịch - Phòng giao dịch số 5 | Chi nhánh Sở giao dịch - Phòng giao dịch số 5Số 22 Thành Công, Phường Thành Công, Quận Ba Đình, Thành phố Hà Nội | (21.0222736, 105.8141352)
- Chi nhánh Láng Hạ - Phòng giao dịch số 07 | Chi nhánh Láng Hạ - Phòng giao dịch số 07Số 106 Đào Tấn, Phường Cống Vị, Quận Ba Đình, Thành phố Hà Nội | (21.034239, 105.80504)
- Chi nhánh Hà Nội I - Phòng giao dịch Số 1 | Chi nhánh Hà Nội I - Phòng giao dịch Số 1Số nhà 124 Kim Mã, phường Kim Mã, quận Ba Đình, thành phố Hà Nội. | (21.032167, 105.826693)
- Chi nhánh Nam Hà Nội - Phòng giao dịch Số 